In [1]:
import random
import os
import pandas as pd 
from datetime import datetime
from tqdm import tqdm
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
import torch.nn as nn
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda', index=0)

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [3]:
train = pd.read_csv('../../DATA/train.csv')
test  = pd.read_csv('../../DATA/test.csv')

In [4]:
# 데이터 타입 변경, 열 이름 변경 

new_column_names = {
    'corporation': 'corp',
    'location': 'loc',
    'supply(kg)': 'supply',
    'price(원/kg)': 'price',
}

train = train.rename(columns=new_column_names)
test = test.rename(columns=new_column_names)

train['timestamp']  = pd.to_datetime(train['timestamp'])
test['timestamp']  = pd.to_datetime(test['timestamp'])

train['newitem'] = train['item'].str.cat([train['corp'], train['loc']], sep=' ')

In [5]:
#가격 시계열 price_data 생성하기 

def before_timeseries_data(train):
    unique_values = train['newitem'].unique()
    # 날짜 범위 생성
    start_date = datetime(2019, 1, 1).date()
    end_date =datetime(2023, 3, 3).date()
    dates = pd.date_range(start_date, end_date, freq='D')

    # 데이터프레임 생성
    price_data = pd.DataFrame(columns=['item'] + dates.strftime('%Y-%m-%d').tolist())
    price_data['item'] = unique_values

    # print(len(price_data)) 39 
    for i in range(len(price_data)) :
        price_data.iloc[i,1:] = train['price'][i*1523 : (i+1)*1523 ]
    price_data['item'] = price_data['item'].astype(str)
    price_data['corp'] = price_data['item'].str[3:4]

    price_data['loca'] = price_data['item'].str[-1:]
    price_data['product'] = price_data['item'].str[:2]
    price_data['product_loca'] =price_data['product'].str.cat(price_data['loca'], sep=' ')
    cols = price_data.columns[-4:]  # 가장 뒤의 3개 열의 열 이름을 선택
    price_data = price_data[cols.tolist() + price_data.columns[:-4].tolist()]  # 열 순서 변경
    
    return price_data

price_data = before_timeseries_data(train)

In [6]:


le = LabelEncoder()
price_data['product_loca'] = le.fit_transform(price_data['product_loca'])
price_df = price_data.copy()

/home/kiwoongyoon/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/kiwoongyoon/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [7]:
class LTSF_Linear(torch.nn.Module):
    def __init__(self, window_size, forecast_size, individual, feature_size):
        super(LTSF_Linear, self).__init__()
        self.window_size = window_size
        self.forecast_size = forecast_size
        self.individual = individual
        self.channels = feature_size
        if self.individual:
            self.Linear = torch.nn.ModuleList()
            for i in range(self.channels):
                self.Linear.append(torch.nn.Linear(self.window_size, self.forecast_size))
        else:
            self.Linear = torch.nn.Linear(self.window_size, self.forecast_size)

    def forward(self, x):
        if self.individual:
            output = torch.zeros([x.size(0), self.forecast_size, x.size(2)],dtype=x.dtype).to(x.device)
            for i in range(self.channels):
                output[:,:,i] = self.Linear[i](x[:,:,i])
            x = output
        else:
            x = self.Linear(x.permute(0,2,1)).permute(0,2,1)
        return x
        
class Data(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __len__(self):
        return len(self.Y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]
    

In [8]:
# Function to reshape the data into a time series format for each ID
def reshape_data(df):
    time_series_data = []
    for idx, row in df.iterrows():
        sales_data = row[5:].values.astype(float)
        time_series_data.append(sales_data)
    return np.array(time_series_data)

# Modified time_slide_df function to work with the current data format
def time_slide_df(data, window_size, forecast_size):
    data_list = []
    dap_list = []
    for idx in range(0, len(data) - window_size - forecast_size + 1):
        x = data[idx:idx + window_size].reshape(window_size, 1)
        y = data[idx + window_size:idx + window_size + forecast_size]
        data_list.append(x)
        dap_list.append(y)
    return np.array(data_list, dtype='float32'), np.array(dap_list, dtype='float32')

# Function to create DataLoader for each ID
def create_dataloader(data, window_size, forecast_size, batch_size):
    X, Y = time_slide_df(data, window_size, forecast_size)
    print(f"{X.shape} is X shape , {Y.shape} is Y shape ")
    ds = Data(X, Y)
    return DataLoader(ds, batch_size=batch_size, shuffle=True)

# Reshape the data
time_series_data = reshape_data(price_df)

In [9]:
# Define the window size, forecast size, and batch size
window_size = 64   # Considering the last 30 days for prediction
forecast_size = 28 # Predicting the next 21 days
batch_size = 2048
epoch_count = 777 #777
lr = 0.001
min_delta = 0.001
patience = 10

In [11]:

future_predictions_by_id = {}
loss_history = {}


    
 

# Iterate through the data by ID
for idx, (id_val, data) in tqdm(enumerate(zip(price_df["item"], time_series_data)), total=len(price_df["item"])):
    # Standardizing the data
    mean_ = np.mean(data)
    std_ = np.std(data)
    if std_ == 0:
        standardized_data = data
    else:
        standardized_data = (data - mean_) / std_
    individual_loss_history = []
    
    # Create DataLoader
    train_dl = create_dataloader(standardized_data, window_size, forecast_size, batch_size)
    best_loss = float('inf')
    no_improvement_count = 0

    # Training the model
    DLinear_model = LTSF_Linear(window_size=window_size, forecast_size=28, individual=False, feature_size=1)
    DLinear_model.to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(DLinear_model.parameters(), lr=lr)
    
    for epoch in range(1, epoch_count + 1):
        loss_list = []
        DLinear_model.train()
        for batch_idx, (data, target) in enumerate(train_dl):
            data, target = data.to(device), target.to(device)  # Move the data to the GPU if available
            optimizer.zero_grad()
            output = DLinear_model(data)
            loss = criterion(output, target.unsqueeze(-1))
            loss.backward()
            optimizer.step()
            loss_list.append(loss.item())
        
        if((epoch % 10) == 0):
            avg_loss = np.mean(loss_list)
            individual_loss_history.append(avg_loss)
            if avg_loss + min_delta < best_loss:
                best_loss = avg_loss
                no_improvement_count = 0
            else:
                no_improvement_count += 1
                if no_improvement_count >= patience:
                    break

    loss_history[id_val] = individual_loss_history

    # Predicting the future 15 days using the last window of data
    last_window_data = torch.tensor(standardized_data[-window_size:]).unsqueeze(0).unsqueeze(-1).float().to(device)
    future_prediction = DLinear_model(last_window_data)

    # Converting the prediction back to the original scale
    if std_ == 0:
        future_prediction = future_prediction.squeeze().detach().cpu().numpy()
    else:
        future_prediction = future_prediction.squeeze().detach().cpu().numpy() * std_ + mean_

    # Store the prediction
    future_predictions_by_id[id_val] = future_prediction

# Future predictions for each ID from 2023-04-05 to 2023-04-25
future_predictions_by_id

  0%|          | 0/39 [00:00<?, ?it/s]

(1432, 64, 1) is X shape , (1432, 28) is Y shape 
(1432, 64, 1) is X shape , (1432, 28) is Y shape 
(1432, 64, 1) is X shape , (1432, 28) is Y shape 
(1432, 64, 1) is X shape , (1432, 28) is Y shape 
(1432, 64, 1) is X shape , (1432, 28) is Y shape 
(1432, 64, 1) is X shape , (1432, 28) is Y shape 
(1432, 64, 1) is X shape , (1432, 28) is Y shape 
(1432, 64, 1) is X shape , (1432, 28) is Y shape 
(1432, 64, 1) is X shape , (1432, 28) is Y shape 
(1432, 64, 1) is X shape , (1432, 28) is Y shape 
(1432, 64, 1) is X shape , (1432, 28) is Y shape 
(1432, 64, 1) is X shape , (1432, 28) is Y shape 
(1432, 64, 1) is X shape , (1432, 28) is Y shape 
(1432, 64, 1) is X shape , (1432, 28) is Y shape 
(1432, 64, 1) is X shape , (1432, 28) is Y shape 
(1432, 64, 1) is X shape , (1432, 28) is Y shape 
(1432, 64, 1) is X shape , (1432, 28) is Y shape 
(1432, 64, 1) is X shape , (1432, 28) is Y shape 
(1432, 64, 1) is X shape , (1432, 28) is Y shape 
(1432, 64, 1) is X shape , (1432, 28) is Y shape 


{'TG A J': array([2782.793  , 1271.4816 , 2553.254  , 2936.0024 , 2647.9763 ,
        3221.0854 , 2937.5098 , 2696.6897 ,  991.25903, 2409.865  ,
        2889.7305 , 2517.2048 , 3221.284  , 3060.9333 , 2850.917  ,
        1358.4973 , 2686.3276 , 3090.848  , 2984.3696 , 3324.4663 ,
        2909.4526 , 2735.6614 , 1280.6957 , 2650.9294 , 3005.1997 ,
        2784.7432 , 3248.7383 , 3179.691  ], dtype=float32),
 'TG A S': array([3889.3862,  992.3489, 3479.042 , 3856.7766, 4177.729 , 4149.8877,
        4246.7695, 4383.6904, 1395.5239, 3944.7124, 4196.5566, 4274.4023,
        4037.5571, 4235.82  , 4510.632 , 1487.8865, 3518.5369, 4562.6953,
        4547.8184, 4144.3066, 4325.493 , 4374.264 , 1456.9377, 4042.0332,
        4471.0244, 4535.4897, 4400.9746, 4816.08  ], dtype=float32),
 'TG B J': array([3030.7468, 1476.0585, 3191.266 , 2307.8594, 3777.0403, 3719.0374,
        3532.2144, 3020.2656,  952.7971, 2745.9453, 3609.8167, 3841.0571,
        4506.1655, 4062.0925, 3221.4912, 1411.2219, 3270

In [12]:
submit = pd.read_csv('./../../DATA/sample_submission.csv')
submit

,ID,answer
0,TG_A_J_20230304,0
1,TG_A_J_20230305,0
2,TG_A_J_20230306,0
3,TG_A_J_20230307,0
4,TG_A_J_20230308,0
...,...,...
1087,RD_F_J_20230327,0
1088,RD_F_J_20230328,0
1089,RD_F_J_20230329,0
1090,RD_F_J_20230330,0


In [13]:
findata = []
for id_val, predictions in future_predictions_by_id.items():
    rounded_predictions = np.round(predictions).astype(np.float64)
    findata.extend(rounded_predictions)

submit['answer'] = findata
submit.loc[submit['answer'] <= 0, 'answer'] = 0
submit.loc[submit.index % 7 == 1, 'answer'] = 0

In [15]:
submit.to_csv('../../DATA/SUBMIT/lstf1.csv', index=False)